In [55]:
import json
import re
import string

import wikipedia
import wptools
from SPARQLWrapper import SPARQLWrapper, JSON
from nltk import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pafou\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [53]:
catType = {"Airports": "?a a dbo:Infrastructure .", "Comics_characters": "?a a dbo:Agent .",
           "Artist": "?a a dbo:Animal .",
           "Astronauts": '?a a dbo:Animal .', "Building": "?a a dbo:Building .", "Astronomical_objects": "",
           "City": "?a a dbo:Place .", "Companies": "?a a dbo:Company .", "Foods": "?a a dbo:Food .",
           "Transport": "?a a dbo:MeanOfTransportation .", "Monuments_and_memorials": "?a a dbo:Place .",
           "Politicians": "?a a dbo:Animal .", "Sports_teams": "?a a dbo:Organisation .",
           "Sportspeople": "?a a dbo:Animal .", "Universities_and_colleges": "?a a dbo:Organisation .",
           "Written_communication": ""}

data = []
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [18]:
k = 10
n = 2

## Exercise 1 :

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
for cat in catType:
    SPARQL_GET_LISTS = f'''
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbc: <http://dbpedia.org/resource/Category:>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dct: <http://purl.org/dc/terms/>

    SELECT DISTINCT ?a ?page ?p WHERE {{
    ?a dct:subject/skos:broader dbc:{cat} .
    {catType[cat]}
    ?a rdfs:label ?label .
    ?a foaf:name ?page .
    ?p foaf:primaryTopic ?a .
    FILTER (lang(?label) = 'en')
    FILTER(!STRSTARTS(?label, "List of"))
    }}limit {k}
    '''
    sparql.setQuery(SPARQL_GET_LISTS)
    try:
        ret = sparql.queryAndConvert()
        for r in ret["results"]["bindings"]:
            name = r["page"]['value']
            url = r["p"]['value']
            url_name = re.findall("([^/]+$)", url)[0]
            page = wikipedia.page(url_name)
            cont = page.content
            number_of_sentences = len(sent_tokenize(cont))
            if number_of_sentences >= n:
                p = wptools.page(url_name, silent=True)
                p.get_parse()
                p.get_wikidata()
                data.append(
                    {"name": name, "url_name": url_name, "txt": page.content, "infobox": p.data["infobox"],
                     "wikidata": p.data["wikidata"],
                     "cat": cat, "url": r["p"]['value']})
    except Exception as e:
        print("ignored")

with open("data.json", "w") as f:
    json.dump(data, f)


## Exercise 2 :


In [39]:
file = open("data.json", "r")
data_to_process = json.load(file)

In [56]:
for item in data_to_process:
    txt = item["txt"]
    text = txt.lower()
    text_p = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text_p)
    filtered_words = [word for word in words if word not in stop_words]
    stemmed = [porter.stem(word) for word in filtered_words]
    pos = pos_tag(stemmed)
    break
